# AHLT - Lab - DDI Baseline

Authors: Ricard Monge (group12) and Cristina Capdevila (group 10)

This notebook contains the deliverables for the AHLT Lab DDI Baseline assignment.
The notebook contains the following sections:

- [Analyze function to compute DependencyParsing](#analyze)
- [Interaction detection function *check_interaction*](#features)
    - [Dependency Tree Analysis Utility functions](#utility)
- [Model comparison on Devel dataset](#dev_table_results)
- [Model comparison on Test dataset](#test_table_results)


<a id="analyze"></a>
## Analyze function to compute DependencyParsing

Function *analyze* takes the sentence text and sends it to an openned instance of the **Stanford CoreNLP Server**, with its wrapper class provided by NLTK. Then the function computes the DependencyGraph object for the text and returns it to be analyzed.

In [ ]:
DependencyParser = CoreNLPDependencyParser(url="http://localhost:9000")

def analyze(s):
    """
    Analyze Text.
    Function which uses an already started Stanford CoreNLP server to analyze
    given text, that is tokenize, tag (Part of Speech tag) and parse.
    Args:
        - text: string with text to analyze.
    Returns:
        - analysis: output of CoreNLP dependency parser.
    """
    # Dependency Parsing
    try:
        analysis, = DependencyParser.raw_parse(s)
    except Exception:
        print(StanfordCoreNLPServer_error)
        exit()
    # Return analysis
    return analysis


<a id='function'></a>
## Interaction detection function *check_interaction*

Given a sentence talking about a pair of entities (drug, drug_n, brand, group), we need to devise features to identify if the sentence states an interaction between those entities and, if it did, of which type mechanism, effect, advise or int(eraction).

To better detect the semantic relations between the entities, we use Stanford CoreNLP parser to compute a Dependency Parsing tree for each sentence, and analyze the relations in the tree with the pair of entities to detect the DDI.

In order to better extract information from the dependency tree we use two utility functions, [see here](#utility). 

Given a list of DDI type clue words, we find in a dependency tree of a sentence the nodes which contain this clue words, and see if any of the entities in a pair is child of the verb node associated with those clue word nodes. 

Using this rules, we find which pairs of entities have one entity hanging from a verb which relates with one of the clue words for a certain DDI type, thus having a higher change of being of that DDI type. If none DDI type matches, we assume it is not a DDI (type *null*).

In [2]:
def check_interaction(analysis, entitities, e1, e2):
    """
    Check Interaction.
    Function to check for interaction of the given pair of entities, by looking
    at the parsed (analysed) sentences which contains it together with all the
    entities in the sentence.
    Args:
        - analysis: DependencyGraph object instance with setnence parsed
            information.
        - entities: dictionary of entities indexed by id with offset as value.
        - e1: string with id of the first entity to consider.
        - e2: string with id of the second entity to consider.
    Return:
        - is_ddi: integer with 0/1 value indicating whether the sentence states
            an interaction between e1 and e2.
        - ddi_type: string with the type of interaction, or null if none.
    """
    is_ddi = False
    ddi_type = "null"

    # Get analysis nodes
    nodes = analysis.nodes
    # Get entities text
    # Split/Lower used to take standard first part of multi-word Drugs
    e1_text = entitities[e1]["text"].split()[0].lower()
    e2_text = entitities[e2]["text"].split()[0].lower()

    # DDI type clues
    advise_clues = ["should", "must", "may", "recommend", "caution"]
    effect_clues = ["produce", "administer", "potentiate", "prevent", "effect"]
    int_clues = ["interact", "interaction"]
    mechanism_clues = ["reduce", "increase", "decrease"]

    # Avoid check if entities are the same or almost the same
    if e1_text == e2_text or e1_text in e2_text or e2_text in e1_text:
        pass

    # Check Advise clues
    elif any(
        is_child(analysis, clue, e1_text) or is_child(analysis, clue, e2_text)
        for clue in get_clue_nodes(nodes, advise_clues)
    ):
        is_ddi = True
        ddi_type = "advise"

    # Check Effect clues
    elif any(
        is_child(analysis, clue, e1_text) or is_child(analysis, clue, e2_text)
        for clue in get_clue_nodes(nodes, effect_clues)
    ):
        is_ddi = True
        ddi_type = "effect"

    # Check Mechanism clues
    elif any(
        is_child(analysis, clue, e1_text) or is_child(analysis, clue, e2_text)
        for clue in get_clue_nodes(nodes, mechanism_clues)
    ):
        is_ddi = True
        ddi_type = "mechanism"

    # Check Int clues
    elif any(
        is_child(analysis, clue, e1_text) or is_child(analysis, clue, e2_text)
        for clue in get_clue_nodes(nodes, int_clues)
    ):
        is_ddi = True
        ddi_type = "int"

    return "1" if is_ddi else "0", ddi_type


<a id="utility"></a>
### Dependency Tree Analysis Utility functions

Utility functions to extract information from dependency tree.
 - Function **is_child** takes a given node from the depdency tree and checks if there is a child which contains the given text.
 
 - Function **get_clue_nodes** takes a given list of clues, finds the nodes in the dependency tree which contain said clues and finds their closest verb ancestors.

In [3]:
def is_child(analysis, node_index, text):
    """
    Check is child.
    Given a DependencyGraph analysis of a sentence, and a node_index for a
    certain node of the graph (or None for the root), this function checks if
    there is a child which contains the given text.
    Args:
        - analysis: DependencyGraph object instance with setnence parsed
            information.
        - node_index: int with node index to check childs from.
        - text: text to look for in child nodes.
    Returns:
        - _: boolean True if text is found in children, False otherwise.
    """
    node = analysis.get_by_address(node_index)
    for triple in analysis.triples(node=node):
        if text in str(triple[2][0]).lower():
            return True
    return False


def get_clue_nodes(nodes, clues):
    """
    Get Clue Nodes.
    Function which gets verb (tag VB*) nodes from which to look for child
    entities. Gets a list of possible clue wordst that indicate a certain type
    of DDI and returns a list with the closest verb ancestors of the nodes
    which contain the clue words.
    Args:
        - nodes: list of current Dependency Graph nodes.
        - clues: list of clue words for a certain type of DDI.
    Returns:
        - _nodes: list of node dictionaries to search child entities from.
    """
    clue_nodes = [n for n in nodes if nodes[n]["lemma"] in clues]
    _nodes = []
    for clue in clue_nodes:
        node = nodes[clue]
        while node["tag"] != "TOP" and "VB" not in node["tag"]:
            node = nodes[node["head"]]
            if not node["tag"]:
                break
        _nodes.append(node["address"])
    return list(set(_nodes))


<a id='dev_table_results'></a>
## Model comparison on Devel dataset

We obtain for the Devel dataset the following metrics:

- Precision: 0.1378
- Recall: 0.6573
- F1: 0.2278

We see that although the rules give a high recall, that is we find most of the DDI with their types, we have a very low precision, that is we categorize as DDI lots of pairs that are not. 

We conclude that the model works well to detect the type of DDI but not well to discern between DDI and non-DDI.

```
SCORES FOR THE GROUP: BASELINE RUN=1
Gold Dataset: /Devel

Partial Evaluation: only detection of DDI (regadless to the type)
tp	fp	fn	total	prec	recall	F1
421	1341	63	484	0.2389	0.8698	0.3749


Detection and Classification of DDI
tp	fp	fn	total	prec	recall	F1
249	1513	235	484	0.1413	0.5145	0.2217


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp	fp	fn	total	prec	recall	F1
99	222	102	201	0.3084	0.4925	0.3793


Scores for ddi with type effect
tp	fp	fn	total	prec	recall	F1
48	379	114	162	0.1124	0.2963	0.163


Scores for ddi with type advise
tp	fp	fn	total	prec	recall	F1
100	744	19	119	0.1185	0.8403	0.2077


Scores for ddi with type int
tp	fp	fn	total	prec	recall	F1
2	168	0	2	0.0118	1	0.0233


MACRO-AVERAGE MEASURES:
	P	R	F1
	0.1378	0.6573	0.2278
________________________________________________________________________

```

<a id='test_table_results'></a>
## Model comparison on Test dataset

We obtain for the Test dataset the following metrics:

- Precision: 0.1246		
- Recall: 0.3704
- F1: 0.1865

We see that compared to the Devel dataset, the recall has decreased but not the precision. All in all, the *F1* score does not decrease much.

We conclude the rule generalises well, although it does not discern well between DDI and non-DDI as shown before.

```
SCORES FOR THE GROUP: BASELINE RUN=2
Gold Dataset: /Test-DDI

Partial Evaluation: only detection of DDI (regadless to the type)
tp	fp	fn	total	prec	recall	F1
778	2636	201	979	0.2279	0.7947	0.3542


Detection and Classification of DDI
tp	fp	fn	total	prec	recall	F1
348	3066	631	979	0.1019	0.3555	0.1584


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp	fp	fn	total	prec	recall	F1
76	232	226	302	0.2468	0.2517	0.2492


Scores for ddi with type effect
tp	fp	fn	total	prec	recall	F1
68	504	292	360	0.1189	0.1889	0.1459


Scores for ddi with type advise
tp	fp	fn	total	prec	recall	F1
184	1902	37	221	0.0882	0.8326	0.1595


Scores for ddi with type int
tp	fp	fn	total	prec	recall	F1
20	428	76	96	0.0446	0.2083	0.0735


MACRO-AVERAGE MEASURES:
	P	R	F1
	0.1246	0.3704	0.1865
________________________________________________________________________
```